# 1. Load data
## 1.1 Ground truth

In [1]:
import os
import csv
import json

import matplotlib.pyplot as plt

In [2]:
frames_true_files = os.listdir('ground_truth/true')
frames_false_files = os.listdir('ground_truth/false')

In [3]:
frames_true = [list(map(int, x.split('.')[0].split('_'))) for x in frames_true_files]
frames_false = [list(map(int, x.split('.')[0].split('_'))) for x in frames_false_files]

In [4]:
data = dict()
for x in frames_true:
    data[f'{x[0]}_{x[1]}'] = {'ground_truth': True}
for x in frames_false:
    data[f'{x[0]}_{x[1]}'] = {'ground_truth': False}
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 1.2 Dane z czujnika z normalizacją min-max

In [5]:
with open('min_max.csv', newline='') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    for id_, value in your_list:
        if id_ in data:
            data[id_]['min_max'] = value

## 1.2 Dane z czujnika z normalizacją min-max

In [6]:
with open('z_score.csv', newline='') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    for id_, value in your_list:
        if id_ in data:
            data[id_]['z_score'] = value

## 1.3 Wyniki inferencji

In [7]:
with open('inference.csv', newline='') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    for id_, value in your_list:
        if id_ in data:
            data[id_]['predicted'] = value

# 2. Sortowanie danych

## 2.1 Utworzenie listy, pominiędzie ramek niezawierajacych wszystkich informacji

In [8]:
data_list = []
for id_, x in data.items():
    if 'z_score' in x and 'min_max' in x and 'predicted' in x and 'ground_truth' in x:
        id_split = id_.split('_')
        video_id = int(id_split[0])
        frame_id = int(id_split[1])
        data_list.append((id_, video_id, frame_id,  float(x['z_score']), float(x['min_max']), x['predicted'] == "True", x['ground_truth']))

print(len(data_list))

2353


In [9]:
import functools


def extract_ids(val):
    key = val[0]
    key_split = key.split('_')
    video_id = int(key_split[0])
    frame_id = int(key_split[1])
    return video_id, frame_id


def compare_id(x, y):
    v_x, f_x = extract_ids(x)
    v_y, f_y = extract_ids(y)
    v_x *= 1000_000
    v_y *= 1000_000
    return (v_x + f_x) - (v_y - f_y)
    # if v_x != v_y:
    #     return v_x > v_y
    # else:
    #     return f_x >= f_y


data_sorted = sorted(data_list, key=lambda key: (key[1], key[2]))
print(data_sorted[:100])

[('0_650', 0, 650, -1.0813170675566173, 0.013333333333333336, False, False), ('0_675', 0, 675, -1.0813170675566173, 0.013333333333333336, False, False), ('0_700', 0, 700, -1.0813170675566173, 0.013333333333333336, False, False), ('0_725', 0, 725, -0.9937692595422224, 0.026666666666666672, False, False), ('0_1600', 0, 1600, -1.0813170675566173, 0.013333333333333336, False, True), ('0_1625', 0, 1625, -0.9937692595422224, 0.026666666666666672, False, False), ('0_1650', 0, 1650, -1.0813170675566173, 0.013333333333333336, False, False), ('0_1675', 0, 1675, -1.0813170675566173, 0.013333333333333336, False, False), ('0_1700', 0, 1700, -1.1688648755710123, 0.0, False, False), ('0_1725', 0, 1725, -1.1688648755710123, 0.0, False, False), ('0_1750', 0, 1750, -1.1688648755710123, 0.0, False, False), ('0_1775', 0, 1775, -1.1688648755710123, 0.0, False, False), ('0_1800', 0, 1800, -1.1688648755710123, 0.0, False, False), ('0_1825', 0, 1825, -1.1688648755710123, 0.0, False, False), ('0_1850', 0, 1850

In [10]:
f = open(f"data.csv", 'w', newline='', encoding='utf-8')
writer = csv.writer(f)
writer.writerow(['id', 'video_id', 'frame_id', 'sensor_z_score', 'sensor_min_max', 'predicted', 'ground_truth'])
writer.writerows(data_sorted)
f.close()

In [12]:
f = open(f"data_village.csv", 'w', newline='', encoding='utf-8')
writer = csv.writer(f)
writer.writerow(['id', 'video_id', 'frame_id', 'sensor_z_score', 'sensor_min_max', 'predicted', 'ground_truth'])
for x in data_sorted:
    if x[1] < 2:
        writer.writerow(x)
f.close()

In [13]:
f = open(f"data_city.csv", 'w', newline='', encoding='utf-8')
writer = csv.writer(f)
writer.writerow(['id', 'video_id', 'frame_id', 'sensor_z_score', 'sensor_min_max', 'predicted', 'ground_truth'])
for x in data_sorted:
    if x[1] >= 2:
        writer.writerow(x)
f.close()